## Double Checking matrix prime calculations for random sampling


In [1]:
# Torch
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch import optim 


# Train + Data 
import sys 
sys.path.append('../Layers')
from Conv1d_NN import * 
from Conv2d_NN import * 
from Conv1d_NN_spatial import *
from Conv2d_NN_spatial import * 

sys.path.append('../Data')
from CIFAR10 import * 


sys.path.append('../Train')
from train2d import * 


/Users/mingikang/miniforge3/envs/ML/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ex1 = torch.rand(1, 5, 5)
print(ex1.shape)

torch.Size([1, 5, 5])


In [3]:
ds_Reg = Conv1d_NN.calculate_distance_matrix(ex1)
print(ds_Reg.shape)

torch.Size([1, 5, 5])


In [4]:
ds_Spatial = Conv1d_NN_spatial.calculate_distance_matrix_N(ex1, ex1)
print(ds_Spatial.shape)

torch.Size([1, 5, 5])


In [5]:
result = torch.equal(ds_Reg, ds_Spatial)


In [6]:
print(result)

False


In [7]:
sub = ds_Reg - ds_Spatial
print(sub)

tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., nan, 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., nan]]])


In [8]:
sm_reg = Conv1d_NN.calculate_similarity_matrix(ex1)

sm_spatial = Conv1d_NN_spatial.calculate_similarity_matrix_N(ex1, ex1)

result = torch.equal(sm_reg, sm_spatial)
print(result)

sub = sm_reg - sm_spatial
print(sub)

True
tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])


## I. Spatial sampling indices for batch processing


In [9]:
x1 = torch.randn(32, 12, 14, 14)
sample_padding = 0
samples = 14

# Generate equally spaced indices for rows (x) and columns (y)
x_ind = torch.round(torch.linspace(sample_padding,
                                   x1.shape[2] - sample_padding - 1,
                                   samples)).to(torch.int)
y_ind = torch.round(torch.linspace(sample_padding,
                                   x1.shape[3] - sample_padding - 1,
                                   samples)).to(torch.int)

# Create a meshgrid of indices
x_grid, y_grid = torch.meshgrid(x_ind, y_ind, indexing='ij')

x_sample = x1[:, :, x_grid, y_grid]
print("x_sample shape: ", x_sample.shape)

x_sample_flatten = x_sample.flatten(start_dim=2)
print("x_sample_flatten shape: ", x_sample_flatten.shape)


flatten = nn.Flatten(start_dim=2)
x2 = flatten(x1)
print("x2 shape: ", x2.shape)

result = torch.equal(x_sample_flatten, x2)
print(result)

x_sample shape:  torch.Size([32, 12, 14, 14])
x_sample_flatten shape:  torch.Size([32, 12, 196])
x2 shape:  torch.Size([32, 12, 196])
True


In [10]:
import torch

# Sample input tensor dimensions [32, 12, 14, 14]
x1 = torch.randn(32, 12, 14, 14)
sample_padding = 0
samples = 14  # We want a 3x3 sample

# Generate equally spaced indices for rows (x) and columns (y)
x_ind = torch.round(torch.linspace(sample_padding,
                                   x1.shape[2] - sample_padding - 1,
                                   samples)).to(torch.int)
y_ind = torch.round(torch.linspace(sample_padding,
                                   x1.shape[3] - sample_padding - 1,
                                   samples)).to(torch.int)

# Create a meshgrid of indices
x_grid, y_grid = torch.meshgrid(x_ind, y_ind, indexing='ij')

# Flatten the grid indices (each will have 'samples*samples' = 9 values)
x_idx_flat = x_grid.flatten()  # Row indices
y_idx_flat = y_grid.flatten()  # Column indices

# To get the corresponding flattened indices for a 14x14 matrix:
width = x1.shape[3]  # This is 14
flat_indices = x_idx_flat * width + y_idx_flat

print("3x3 Sample pixel positions (flattened indices):", flat_indices)

flatten = nn.Flatten(start_dim=2)
x1 = torch.randn(32, 12, 14, 14)
x2 = flatten(x1)
print(x2.shape)



3x3 Sample pixel positions (flattened indices): tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169

# I. Adding Spatial Coordinate channels -> + 2 channels for x coord & y coord

In [11]:
import torch

# Example tensor of shape [18, 3, 32, 32]
x = torch.randn(18, 3, 32, 32)

# Create 2 extra channels of ones with shape [18, 2, 32, 32]
ones_channels = torch.ones(x.size(0), 2, x.size(2), x.size(3), device=x.device, dtype=x.dtype)

# Concatenate along the channel axis (dim=1)
x_with_extra = torch.cat([x, ones_channels], dim=1)

print(x_with_extra.shape)  # Expected shape: [18, 5, 32, 32]

torch.Size([18, 5, 32, 32])


In [12]:
import torch

# Assume x_with_extra is your [18, 5, 32, 32] tensor.
# Method 1: Using a boolean equal check
if torch.all(x_with_extra[:, -2:, :, :] == 1):
    print("The last two channels are filled with 1s.")

# Method 2: Using torch.allclose for floating point comparisons
ones_tensor = torch.ones_like(x_with_extra[:, -2:, :, :])
if torch.allclose(x_with_extra[:, -2:, :, :], ones_tensor):
    print("The last two channels are filled with 1s.")

The last two channels are filled with 1s.
The last two channels are filled with 1s.


In [13]:
print("Last channel values:")
print(x_with_extra[:, -1, :, :])

Last channel values:
tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        ...,

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
 

In [14]:
import torch

# 2 spatial channels of x coord and y coord
x_ind = torch.arange(0, 32)
y_ind = torch.arange(0, 32)

x_grid, y_grid = torch.meshgrid(x_ind, y_ind, indexing='ij')

print("x_grid.shape: ", x_grid.shape)
print("y_grid.shape: ", y_grid.shape)

ex = torch.randn(18, 3, 32, 32)

# Expand x_grid and y_grid to have a batch dimension
x_grid_expanded = x_grid.unsqueeze(0).expand(ex.size(0), -1, -1)  # Shape: [18, 32, 32]
y_grid_expanded = y_grid.unsqueeze(0).expand(ex.size(0), -1, -1)  # Shape: [18, 32, 32]

# Add a channel dimension
x_grid_channeled = x_grid_expanded.unsqueeze(1)  # Shape: [18, 1, 32, 32]
y_grid_channeled = y_grid_expanded.unsqueeze(1)  # Shape: [18, 1, 32, 32]



# Concatenate along the channel dimension
x_spatial_channel = torch.cat([ex, x_grid_channeled, y_grid_channeled], dim=1)
print("Final shape:", x_spatial_channel.shape)  # Expected shape: [18, 5, 32, 32]

print(x_spatial_channel[-1, -1, :, :].shape)

x_grid.shape:  torch.Size([32, 32])
y_grid.shape:  torch.Size([32, 32])
Final shape: torch.Size([18, 5, 32, 32])
torch.Size([32, 32])


In [ ]:
def coordinate_channels(tensor_shape):
    x_ind = torch.arange(0, tensor_shape[2])
    y_ind = torch.arange(0, tensor_shape[3])
    
    x_grid, y_grid = torch.meshgrid(x_ind, y_ind, indexing='ij')
    
    x_grid = x_grid.float().unsqueeze(0).expand(tensor_shape[0], -1, -1).unsqueeze(1)
    y_grid = y_grid.float().unsqueeze(0).expand(tensor_shape[0], -1, -1).unsqueeze(1)
    
    xy_grid = torch.cat((x_grid, y_grid), dim=1)
    xy_grid_normalized = F.normalize(xy_grid, p=2, dim=1)
    return xy_grid_normalized

In [27]:
ex = torch.randn(18, 3, 32, 32)
xy_grid = coordinate_channels(ex.shape)


IPython -- An enhanced Interactive Python

IPython offers a fully compatible replacement for the standard Python
interpreter, with convenient shell features, special commands, command
history mechanism and output results caching.

At your system command line, type 'ipython -h' to see the command line
options available. This document only describes interactive features.

GETTING HELP
------------

Within IPython you have various way to access help:

  ?         -> Introduction and overview of IPython's features (this screen).
  object?   -> Details about 'object'.
  object??  -> More detailed, verbose information about 'object'.
  %quickref -> Quick reference of all IPython specific syntax and magics.
  help      -> Access Python's own help system.

If you are in terminal IPython you can quit this screen by pressing `q`.


MAIN FEATURES
-------------

* Access to the standard Python help with object docstrings and the Python
  manuals. Simply type 'help' (no quotes) to invoke it.

* Ma

In [28]:
print(xy_grid.shape)

torch.Size([18, 2, 32, 32])


In [29]:
print(xy_grid[-1, -1, 1, :])

tensor([0.0000, 0.7071, 0.8944, 0.9487, 0.9701, 0.9806, 0.9864, 0.9899, 0.9923,
        0.9939, 0.9950, 0.9959, 0.9965, 0.9971, 0.9975, 0.9978, 0.9981, 0.9983,
        0.9985, 0.9986, 0.9988, 0.9989, 0.9990, 0.9991, 0.9991, 0.9992, 0.9993,
        0.9993, 0.9994, 0.9994, 0.9994, 0.9995])


## Batch normalization + transposed convolutional layer

In [ ]:
nn.BatchNorm2d